# Measure precision

* Find other samples that share the same active bits in the binary domain for the top features
* Measure proportion of such samples which black box model is aligned with the given instance

In [88]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import cdist
import sklearn
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import classification_report

class Binarizer:
    
    def __init__(self, training_data, feature_names=None,
                 categorical_feature_idxes=None,
                 qs=[25, 50, 75], **kwargs):
        """
        Args:
            training_data (np.ndarray): Training data to measure training data statistics
            feature_names (list): List of feature names
            categorical_feature_idxes (list): List of idxes of features that are categorical
            qs (list): Discretization bins

        Assumptions:
            * Data only contains categorical and/or numerical data
            * Categorical data is already converted to ordinal labels (e.g. via scikit-learn's
                OrdinalEncoder)

        """
        self.training_data = training_data
        self.num_features = self.training_data.shape[1]

        # Parse columns
        if feature_names is not None:
            # TODO input validation
            self.feature_names = list(feature_names)
        else:
            self.feature_names = list(range(self.num_features))
        self.categorical_feature_idxes = categorical_feature_idxes
        if self.categorical_feature_idxes:
            self.categorical_features = [self.feature_names[i] for i in
                                         self.categorical_feature_idxes]
            self.numerical_features = [f for f in self.feature_names if f not in self.categorical_features]
            self.numerical_feature_idxes = [idx for idx in range(self.num_features) if
                                            idx not in self.categorical_feature_idxes]
        else:
            self.categorical_features = []
            self.numerical_features = self.feature_names
            self.numerical_feature_idxes = list(range(self.num_features))

        # Some book-keeping: keep track of the original indices of each feature
        self.dict_num_feature_to_idx = {feature: idx for (idx, feature) in
                                        enumerate(self.numerical_features)}
        self.dict_feature_to_idx = {feature: idx for (idx, feature) in
                                    enumerate(self.feature_names)}
        self.list_reorder = [self.dict_feature_to_idx[feature] for feature in
                             self.numerical_features + self.categorical_features]

        # Get training data statistics
        # Numerical feature statistics
        if self.numerical_features:
            training_data_num = self.training_data[:, self.numerical_feature_idxes]
            self.sc = StandardScaler(with_mean=False)
            self.sc.fit(training_data_num)
            self.qs = qs
            self.all_bins_num = np.percentile(training_data_num, self.qs, axis=0).T

        # Categorical feature statistics
        if self.categorical_features:
            training_data_cat = self.training_data[:, self.categorical_feature_idxes]
            self.dict_categorical_hist = {
                feature: np.bincount(training_data_cat[:, idx]) / self.training_data.shape[0] for
                (idx, feature) in enumerate(self.categorical_features)
            }

        # Another mapping fr om feature to type
        self.dict_feature_to_type = {
            feature: 'categorical' if feature in self.categorical_features else 'numerical' for
            feature in self.feature_names}
        
    def discretize(self, X, qs=[25, 50, 75], all_bins=None):
        if all_bins is None:
            all_bins = np.percentile(X, qs, axis=0).T
        return (np.array([np.digitize(a, bins)
                          for (a, bins) in zip(X.T, all_bins)]).T, all_bins)

    def fetch_similar(self, data_row, test_data, feature_idxes):
        """
        Fetch data from test_data which binarized features match those of data_row
        """
        # Scale the data
        data_row = data_row.reshape((1, -1))

        # Split data into numerical and categorical data and process
        list_disc = []
        if self.numerical_features:
            data_num = data_row[:, self.numerical_feature_idxes]            
            test_data_num = test_data[:, self.numerical_feature_idxes]
            
            data_num = np.concatenate((data_num, test_data_num))
            
            # Discretize
            data_synthetic_num_disc, _ = self.discretize(data_num, self.qs,
                                                         self.all_bins_num)
            list_disc.append(data_synthetic_num_disc)

        if self.categorical_features:
            # Sample from training distribution for each categorical feature
            data_cat = data_row[:,self.categorical_feature_idxes]
            test_data_cat = test_data[:, self.categorical_feature_idxes]
            data_cat = np.concatenate((data_cat, test_data_cat))
            
            list_disc.append(data_cat)

        # Concatenate the data and reorder the columns
        data_synthetic_disc = np.concatenate(list_disc, axis=1)
        data_synthetic_disc = data_synthetic_disc[:, self.list_reorder]
        
        data_instance_disc = data_synthetic_disc[0]
        test_data_disc = data_synthetic_disc[1:]
        
        # Fetch neighbors from real test data where top features are the same
        same_features = np.where(np.all(test_data_disc[:, feature_idxes] == 
                                        data_instance_disc[feature_idxes], axis=1))[0]
        similar_neighbors = test_data[same_features]
        return similar_neighbors

## Try measuring precision of lime tabular explainer

In [49]:
from lime.lime_tabular import LimeTabularExplainer

df = pd.read_csv('../data/german_credit_data.csv')
print(df.shape)
df = df.fillna('None')
target_col = 'Risk'
df[df[target_col] == 'good'][target_col] = 1
df[df[target_col] == 'bad'][target_col] = 0

print(df[target_col].value_counts())

numerical_features = ['Age', 'Credit amount', 'Duration']
categorical_features = ['Sex', 'Job', 'Housing', 'Saving accounts', 
                        'Checking account', 'Purpose']
feature_names = list(df.columns)[:-1]
X, y = df[df.columns[:-1]], df[target_col]

dict_le = {}
for cat_col in categorical_features:
    le = LabelEncoder()
    X[cat_col] = le.fit_transform(X[cat_col])
    dict_le[cat_col] = le

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
clf = RandomForestClassifier(n_estimators=100, max_depth=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(classification_report(clf.predict(X_test), y_test))

(1000, 10)
good    700
bad     300
Name: Risk, dtype: int64


/experiments/venv_faster_lime/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/experiments/venv_faster_lime/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/experiments/venv_faster_lime/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

0.67
              precision    recall  f1-score   support

         bad       0.11      0.80      0.20         5
        good       0.98      0.66      0.79        95

    accuracy                           0.67       100
   macro avg       0.55      0.73      0.49       100
weighted avg       0.94      0.67      0.76       100



In [44]:
df.columns

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose', 'Risk'],
      dtype='object')

In [32]:
explainer = LimeTabularExplainer(training_data=X_train.to_numpy(), 
                                 feature_names=feature_names,
                                 categorical_features=[idx for (idx, col) in enumerate(df.columns) if col in categorical_features],
                                 categorical_names=categorical_features)

exp = explainer.explain_instance(
    data_row=X_test.to_numpy()[0],
    predict_fn=clf.predict_proba,
    labels=(0,1),
    num_features=3
)

In [33]:
sorted(exp.as_list(1), key=lambda x: x[1], reverse=True)

[('Duration <= 12.00', 0.12229827313833372),
 ('Age > 42.00', 0.0414974037456646),
 ('Checking account=1', -0.14738513434575204)]

In [34]:
features_exp = list(map(lambda x: x[0], exp.as_list(1)))
features_exp

['Checking account=1', 'Duration <= 12.00', 'Age > 42.00']

In [35]:
features_used = []
features_idx = []
for feature_exp in features_exp:
    for idx, f in enumerate(feature_names):
        if f in feature_exp:
            features_used.append(f)
            features_idx.append(idx)
            break            

            
features_idx = sorted(features_idx)
print(features_used)
print(features_idx)

['Checking account', 'Duration', 'Age']
[0, 5, 7]


In [89]:
binarizer = Binarizer(training_data=X_train.to_numpy(),
                      feature_names=feature_names,
                      categorical_feature_idxes=[idx for (idx, col) in enumerate(feature_names) 
                                                 if col in categorical_features])

In [91]:
similar = binarizer.fetch_similar(
    data_row=X_test.to_numpy()[0],
    test_data=X_test.to_numpy(),
    feature_idxes=features_idx
)
similar.shape

(14, 9)

In [94]:
print('Test precision: {:.2f}'.format(np.mean(clf.predict(similar) == 
                                               clf.predict(X_test.to_numpy()[0].reshape(1, -1)))))
print('Test coverage: {:.2f}'.format(similar.shape[0] / X_test.shape[0]))

Test precision: 1.00
Test coverage: 0.14
